In [1]:
from google.colab import drive
drive.mount('/content/drive')

ModuleNotFoundError: No module named 'google'

In [19]:
import sys
sys.path.append('/content/drive/MyDrive/activelearning')

In [20]:
from Topic_Models.topic_model import Topic_Model
import pandas as pd
from flask_app.classifier import Active_Learning
from sklearn.feature_extraction.text import TfidfVectorizer
from Topic_Models.Neural_Topic_Model import Neural_Model
import pickle
from multiprocessing import Process, Manager
import copy
from scipy.sparse import csr_matrix
import numpy as np
from scipy.sparse import csr_matrix, hstack

In [3]:
'''
The processed document path and the raw documents path
'''

doc_dir = './Topic_Models/Data/newsgroups/newsgroup_test.json'
processed_doc_dir = './Topic_Models/Data/newsgroups/newsgroup_test_processed.pkl'

'''
Mapping the mode numbers to which model we use
LA: active learning baseline
'''
model_types_map = {0: 'LA' , 1: 'LDA', 2: 'SLDA', 3: 'ETM', 4: 'CTM', 7: 'LLDA', 8: 'PLDA', 9: 'Bertopic', 10: 'SCTM'}

'''
num_iter: number of iterations to run when updating sLDA model
load_data: If use load_data, a trained topic model would be loaded.
'''
num_iter = 1200
load_data = True
save_model = False

'''
Enter the number of topics for the model you just trained
'''
num_topics =16
inference_alg = 'logreg'
test_dataset_name = './Topic_Models/Data/newsgroups/newsgroup_train.json'
test_processed_doc_dir = './Topic_Models/Data/newsgroups/newsgroup_train_processed.pkl'

'''
Keep those and don't change
'''
USE_TEST_DATA = True
USE_PROCESSED_TEXT = False
CONCATENATE_KEYWORDS = True
table = pd.read_json(doc_dir)
training_length = len(table)
REGRESSOR_PREDICT = True
mode = 2

In [4]:
model = Topic_Model(num_topics, 0, model_types_map[mode], processed_doc_dir, training_length, {}, True, './Topic_Models/Model/{}_{}.pkl'.format(model_types_map[mode], num_topics))
# model = Neural_Model('./Model/ETM_{}.pkl'.format(num_topics), processed_doc_dir, doc_dir)

In [5]:
'''
find_doc_for_TA: Given a dictionary, where the keys are topic numbers
the values are list of tuples of (document_id, probability), first find
the topic that has the maximum median probability, then finds the document
within that topic that has the maximum probaiblity
'''
from flask_app.utils.tools import find_doc_for_TA
import numpy as np

In [6]:
'''
The document topic distribution for the topic model
'''
document_probas = model.document_probas

In [7]:
'''
The result is a N X K matrix. N is the number of
documents, K is the number of topics
'''
np.shape(model.doc_topic_probas)

(7049, 16)

In [8]:
def count_docs(dictionary):
    result = 0
    for k, v in dictionary.items():
        result += len(v)

    return result

def update_median_prob(topic_num, idx_in_topic, doc_probs):
    try:
        doc_probs[topic_num].pop(idx_in_topic)
    except:
          doc_probs.pop(topic_num)

def print_topic_lens(dictionary):
    for k, v in dictionary.items():
        print('topic {} num docs {}'.format(k, len(v)))

In [9]:
%%capture
print('total num docs', count_docs(document_probas))

recommended_docs = []
recommended_topics = []
# Since the length of the document probas change, np.ones length should also change
for i in range(100):
    chosen_idx, chosen_topic, chosen_idx_in_topic = find_doc_for_TA(document_probas, np.ones(training_length))
    update_median_prob(chosen_topic, chosen_idx_in_topic, document_probas)
    # print_topic_lens(document_probas)
    print('doc id is ', chosen_idx)
    # chosen_topic
    recommended_topics.append(chosen_topic)
    recommended_docs.append(chosen_idx)

In [10]:
'''
Run a hundred iterations to pick the document that has the maximum median probability in the topic.
Check the distribution of topics being picked. This is a good way to check topic diversity first
'''

from collections import Counter
Counter(recommended_topics)

Counter({1: 24, 8: 33, 7: 43})

In [11]:
import numpy as np
def save_results(lst, file_name):
    a_obj = np.array(lst, dtype=object)
    np.save(file_name, a_obj)

In [13]:
'''
Active learning baseline to test the metrics of clf without topic modeling
'''
def calculate_activelearning_diversity(process=False, unigram=False):
    accs, purity, ri, nmi = [], [], [], []
    test_accs, test_purity, test_ri, test_nmi = [], [], [], []
    test_df = pd.read_json(test_dataset_name)
    # print('test length is', len(test_df))

    '''
    Read the documents and the labels
    '''
    df = pd.read_json(doc_dir)
    labels = df.sub_labels.values.tolist()

    raw_texts = df.text.values.tolist()[0:training_length]
    test_texts = test_df.text.values.tolist()
    # raw_texts.extend(test_texts)


    '''
    Tried to test the performance of unigram and bigrams, the performance of bigram vectorizer is better
    '''
    if unigram == True:
        vectorizer = TfidfVectorizer(stop_words='english', lowercase=True, ngram_range=(1, 1))
    else:
        vectorizer = TfidfVectorizer(stop_words='english', lowercase=True, ngram_range=(1, 2))
    '''
    If process is True, load the processed data and use processed data as inputs for the clf
    else, just use the raw texts to as input for the clf. Results indicate that using raw texts
    has higher performance than using processed data
    '''
    if not process:
        vectorizer_idf = vectorizer.fit_transform(raw_texts)
        test_vecorizer = vectorizer.transform(test_texts)
    else:
        with open(processed_doc_dir, 'rb') as inp:
            loaded_train_data = pickle.load(inp)
            processed_train_data = loaded_train_data['datawords_nonstop']


        text_data = [' '.join(doc) for doc in processed_train_data]
        # from sklearn.feature_extraction.text import CountVectorizer
        # vectorizer = CountVectorizer(binary=True)
        vectorizer_idf = vectorizer.fit_transform(text_data)


    '''
    Initialize the active learning classifier
    '''
    alto = Active_Learning(raw_texts, None,  None, df, inference_alg, vectorizer_idf, training_length, 0, test_df, test_vecorizer)

    recommended_docs1 = []
    recommended_topics1 = []
    scores = []

    '''
    Label 300 hundred documents and save the three metrics
    '''
    for i in range(len(df)):
        '''
        Get the recommended document id and the score (entropy) for the current document
        '''
        recommend_id, score = alto.recommend_document(True)

        alto.label(recommend_id, labels[recommend_id])
        recommended_docs1.append(recommend_id)
        recommended_topics1.append(alto.last_recommended_topic)
        scores.append(score)
        # print(raw_texts[recommend_id])

        '''
        clf only works for at least 2 distinct classes
        '''
        if len(alto.classes) >= 3:
            train_acc, b, p, r, n, e, f, g = alto.eval_classifier()
            accs.append(train_acc)
            purity.append(p)
            ri.append(r)
            nmi.append(n)
            test_accs.append(b)
            test_purity.append(e)
            test_ri.append(f)
            test_nmi.append(g)

    return accs, purity, ri, nmi, test_accs, test_purity, test_ri , test_nmi

In [21]:
'''
Evaluate the performance of activalearning clf with topic model features added to the clf
'''
def calculate_topic_diversity(module, concat, topic_features_only, Model=None):
    model_topics = []
    accs, purity, ri, nmi = [], [], [], []
    test_accs, test_purity, test_ri, test_nmi = [], [], [], []
    test_df = pd.read_json(test_dataset_name)
    # print('test length is', len(test_df))

    if Model is None:
        if module == 3:
            model = Neural_Model('./Topic_Models/Model/ETM_{}.pkl'.format(num_topics), processed_doc_dir, doc_dir)
        elif module == 4:
            model = Neural_Model('./Topic_Models/Model/CTM_{}.pkl'.format(num_topics), processed_doc_dir, doc_dir)
        elif module == 9:
            model = Neural_Model('./Topic_Models/Model/Bertopic_{}.pkl'.format(num_topics), processed_doc_dir, doc_dir)
        elif module == 10:
            model = Neural_Model('./Topic_Models/Model/SCTM_{}.pkl'.format(num_topics), processed_doc_dir, doc_dir)
        else:
            model = Topic_Model(num_topics, 0, model_types_map[module], processed_doc_dir, training_length, {}, True, './Topic_Models/Model/{}_{}.pkl'.format(model_types_map[module], num_topics))
    else:
        '''
        If no module is selected, then use the Model passed in in the parameter
        '''
        model = Model

    '''
    Read texts, construct a text encoding for the clf
    '''
    df = pd.read_json(doc_dir)
    labels = df.sub_labels.values.tolist()
    raw_texts = df.text.values.tolist()[0:training_length]
    document_probas, doc_topic_probas = model.group_docs_to_topics()
    vectorizer = TfidfVectorizer(stop_words='english', lowercase=True, ngram_range=(1,1))
    test_texts = test_df.text.values.tolist()

    with open(test_processed_doc_dir, 'rb') as inp:
        loaded_test_data = pickle.load(inp)
        test_processed_data = loaded_test_data['datawords_nonstop']

    '''
    If concatenation is true, concatenate the topic distribution with the text features
    '''
    if concat == True:
        if topic_features_only == True:
            vectorizer_idf = csr_matrix(model.doc_topic_probas).astype(np.float64)
        else:
            vectorizer_idf = vectorizer.fit_transform(raw_texts)

            '''
            concatenate the features from topic model with the clf
            '''
            concatenated_features = model.concatenate_features(model.doc_topic_probas, vectorizer_idf)

            vectorizer_idf = concatenated_features

        # test_vectorizer = vectorizer.transform(test_texts)



        # test_document_prob_dist = model.predict_topic_distribution_unseen_doc(test_processed_data)
        
        # test_vecotizer_concatenated_features = model.concatenate_features(test_document_prob_dist, test_vectorizer)

        # test_vecotizer_concatenated_features = csr_matrix(test_document_prob_dist).astype(np.float64)
        
        test_vecotizer_concatenated_features = None
    else:
        model_concate_texts = raw_texts
        vectorizer_idf = vectorizer.fit_transform(model_concate_texts)



    alto = Active_Learning(raw_texts, copy.deepcopy(document_probas), doc_topic_probas, df, inference_alg, vectorizer_idf, training_length, 1, test_df, test_vecotizer_concatenated_features)

    print('start synthetic labeling')
    recommended_docs1 = []
    recommended_topics1 = []
    scores = []

    '''
    Label 300 documents
    '''
    for i in range(700):
        recommend_id, score = alto.recommend_document(True)



        alto.label(recommend_id, labels[recommend_id])
        recommended_docs1.append(recommend_id)
        recommended_topics1.append(alto.last_recommended_topic)
        scores.append(score)
        # print(raw_texts[recommend_id])
        # print(alto.user_labels)
        if len(alto.classes) >= 3:
            train_acc, b, p, r, n, e, f, g = alto.eval_classifier()
            accs.append(train_acc)
            purity.append(p)
            ri.append(r)
            nmi.append(n)
            test_accs.append(b)
            test_purity.append(e)
            test_ri.append(f)
            test_nmi.append(g)

        '''
        Update and retrain supervised LDA based on available labels. set i%N == 0, where N
        is the number of documents you labled before you want to update sLDA
        '''
        if module == 2 and i != 0 and i % 20 == 0:
            # pred_labels = alto.classifier.predict(alto.text_vectorizer)
            # user_labels = {i:pred_labels[i] for i in range(len(pred_labels))}
            model = Topic_Model(num_topics, num_iter, model_types_map[module], processed_doc_dir, training_length, alto.user_labels, False, None)
            model.train('./Topic_Models/Model/SLDA_test.pkl')
            model = Topic_Model(num_topics, 0, model_types_map[module], processed_doc_dir, training_length, {}, True, './Topic_Models//Model/SLDA_test.pkl')
            document_probas, doc_topic_probas = model.group_docs_to_topics()
            alto.update_doc_probs(copy.deepcopy(document_probas), doc_topic_probas)
            vectorizer_idf = vectorizer.fit_transform(raw_texts)
            '''
            concatenate the features from topic model with the classifier
            '''
            # concatenated_features = model.concatenate_features(vectorizer_idf)
            concatenated_features = model.concatenate_features(model.doc_topic_probas, vectorizer_idf)
            


            vectorizer_idf = concatenated_features
            alto.update_text_vectorizer(vectorizer_idf)
            


    return recommended_topics1, accs, purity, ri, nmi, test_accs, test_purity, test_ri , test_nmi

In [15]:
'''
Given a matrix, make its rows consistent length
'''
def make_consistent_rows(matrix):
    # Find the length of the shortest row
    min_length = min(len(row) for row in matrix)

    # Trim each row to the length of the shortest row
    a_uniform = [row[-min_length:] for row in matrix]
    a_np = np.array(a_uniform)
    return a_np

In [16]:
def run_experiment_and_save(module, save_path, topic_features_only=True):
  '''
  module: model_types_map = {0: 'LA' , 1: 'LDA', 2: 'SLDA', 3: 'ETM', 4: 'CTM', 7: 'LLDA', 8: 'PLDA', 9: 'Bertopic'}
  Enter the module number to run the the model
  '''

  if module == 0:
    start_time = time.time()
    a0, p0, r0, n0, ta0, tp0, tr0, tn0 = calculate_activelearning_diversity(False, True)
    end_time = time.time()
    result_lst = [a0, p0, r0, n0, ta0, tp0, tr0, tn0, [[end_time-start_time]]]
    save_results(result_lst, save_path)
  else:
    start_time = time.time()
    recommended_topics1, a1, p1, r1, n1, ta1, tp1, tr1, tn1 = calculate_topic_diversity(module, True, topic_features_only)
    end_time = time.time()
    result_lst = [a1, p1, r1, n1, ta1, tp1, tr1, tn1, [[end_time-start_time]]]
    print('purity list length is ', len(p1))
    save_results(result_lst, save_path)


In [17]:
'''
Run the experiment over n runs then take the medium of each run to 
save
'''
import time
def run_experiment_over_n_runs(module, save_path, n_runs):
     a, p, r, n, ta, tp, tr, tn = [], [], [], [], [], [], [], []

     running_time = []
     if module == 0:
          for i in range(n_runs):
               start_time = time.time()
               a0, p0, r0, n0, ta0, tp0, tr0, tn0 = calculate_activelearning_diversity(False, True)
               end_time = time.time()
               a.append(a0)
               p.append(p0)
               r.append(r0)
               n.append(n0)
               ta.append(ta0)
               tp.append(tp0)
               tr.append(tr0)
               tn.append(tn0)
               running_time.append(end_time-start_time)
     else:
          for i in range(n_runs):
               start_time = time.time()
               recommended_topics1, a1, p1, r1, n1, ta1, tp1, tr1, tn1 = calculate_topic_diversity(module, True)
               end_time = time.time()

               a.append(a1)
               p.append(p1)
               r.append(r1)
               n.append(n1)
               ta.append(ta1)
               tp.append(tp1)
               tr.append(tr1)
               tn.append(tn1)
               running_time.append(end_time-start_time)

     a = make_consistent_rows(a)
     p = make_consistent_rows(p)
     r = make_consistent_rows(r)
     n = make_consistent_rows(n)
     ta = make_consistent_rows(ta)
     tp = make_consistent_rows(tp)
     tr = make_consistent_rows(tr)
     tn = make_consistent_rows(tn)

     result_lst = [a, p, r, n, ta, tp, tr, tn, [running_time]]
     save_results(result_lst, save_path)

In [16]:
%%capture
'''
Run the baseline experiment. The saved results can be plotted in the new_mode_plot.ipynb notebook
'''
run_experiment_and_save(0, './model_testing_results/baseline_test_docs_regularized_one_run.npy')
# run_experiment_over_n_runs(0, './model_testing_results/baseline_test_docs_5_runs_med.npy', 5)

In [17]:
%%capture
'''
Run the LDA experiment. The saved results can be plotted in the new_mode_plot.ipynb notebook
'''
run_experiment_and_save(1, './model_testing_results/16_topics_LDA_test_docs_regularized_one_run.npy', False)
# run_experiment_over_n_runs(1, './model_testing_results/LDA_500_docs_5_runs.npy', 5)

In [22]:
# %%capture
'''
Run the sLDA experiment
'''
run_experiment_and_save(2, './model_testing_results/16_topics_SLDA_test_docs_update_regularized_one_run.npy', False)
# run_experiment_over_n_runs(2, './model_testing_results/SLDA_500_docs_5_runs.npy', 5)

start synthetic labeling
picked doc is  2311
[]
picked doc is  4609
['med']
max topic index is  1
max median topic is  1
['med', 'politics.mideast']
[[ 202   96    1]
 [1180  623   44]
 [ 126  197   43]
 [1230  166   59]
 [ 982  479   12]
 [ 108  272    2]
 [ 874  344    9]]
train acc 0.06766917293233082; purity 0.27124414810611436; RI 0.4915520012418342; NMI 0.04750825167588831
test acc -1; purity -1; RI -1; NMI -1
max topic index is  1
max median topic is  1
['med', 'politics.mideast', 'windows.x']
[[ 202    1   96    0]
 [1176   39  629    3]
 [ 127   42  197    0]
 [1232   57  165    1]
 [ 982   12  479    0]
 [ 108    2  272    0]
 [ 872    9  346    0]]
train acc 0.06752730883813307; purity 0.2725209249538942; RI 0.49156238743261255; NMI 0.04772597671049312
test acc -1; purity -1; RI -1; NMI -1
max topic index is  1
max median topic is  1
['med', 'politics.mideast', 'windows.x', 'os.ms-windows.misc']
[[ 202    0   96    1    0]
 [1166   40  637    3    1]
 [ 128   39  197    2   

In [38]:
%%capture
'''
Run the CTM experiment
'''
run_experiment_and_save(4, './model_testing_results/16_topics_CTM_test_docs_regularized_one_run.npy', False)
# run_experiment_over_n_runs(4, './model_testing_results/CTM_500_docs_5_runs_16topics.npy', 5)

In [37]:
%%capture
'''
Run the CTM experiment
'''
run_experiment_and_save(10, './model_testing_results/16_topics_SCTM_test_docs_regularized_one_run.npy', False)
# run_experiment_over_n_runs(4, './model_testing_results/CTM_500_docs_5_runs_16topics.npy', 5)

In [28]:
%%capture
'''
Run the ETM experiment
'''
run_experiment_and_save(3, './model_testing_results/16_topics_ETM.npy')

FileNotFoundError: [Errno 2] No such file or directory: './Topic_Models/Model/ETM_16.pkl'

In [17]:
%%capture
'''
Run the CTM experiment
'''
run_experiment_and_save(4, './model_testing_results/50_topics_CTM_topic_features_only.npy')

In [30]:
%%capture
'''
Run the PLDA experiment
'''
run_experiment_and_save(8, './model_testing_results/16_topics_PLDA.npy')

KeyboardInterrupt: 

In [21]:
%%capture
'''
Run the Bertopic experiment
'''
run_experiment_and_save(9, './model_testing_results/50_topics_BERTOPIC_topic_features_only.npy')

### Fetch word priors from the topic model. Possibly can be used as features for the classifier

In [ ]:
from topic_model import Topic_Model
full_len = len(pd.read_json('./Data/congressional_bill_train.json'))
Model = Topic_Model(13, 2500, 'LDA', './Data/congressional_bill_train_processed.pkl', full_len , {}, False, None)
save_path = './Model/{}_{}.pkl'.format('LDA', 13)
Model.train(save_path)

ModuleNotFoundError: ignored

In [ ]:
len(Model.model.used_vocabs)

9306

In [ ]:
Model.model.used_vocabs

['program', 'united', 'national', 'establish', 'authorize', 'code', 'section', 'title', 'public', 'state', 'health', 'service', 'require', 'housing', 'secretary', 'development', 'relate', 'energy', 'assistance', 'education', 'tax', 'use', 'increase', 'law', 'grant', 'income', 'year', 'revenue', 'improve', 'respect', 'extend', 'land', 'fund', 'individual', 'security', 'internal', 'system', 'prohibit', 'provision', 'department', 'loan', 'control', 'water', 'school', 'commission', 'amend', 'care', 'agency', 'project', 'area', 'facility', 'agricultural', 'research', 'benefit', 'provide', 'construction', 'local', 'person', 'government', 'protection', 'promote', 'transportation', 'labor', 'employee', 'administration', 'authority', 'additional', 'conservation', 'protect', 'permit', 'assist', 'employment', 'shall', 'commerce', 'payment', 'safety', 'community', 'establishment', 'new', 'insurance', 'appropriation', 'drug', 'right', 'requirement', 'child', 'order', 'information', 'interstate', 'd

In [ ]:
for word in Model.model.used_vocabs:
    prior = Model.model.get_word_prior(word)
    if len(prior) > 0:
        print(word)

In [ ]:
Model.model.tw

0

In [ ]:
model_vocabs = set()
for i in Model.data_words_nonstop:
    for j in i:
        model_vocabs.add(j)

len(model_vocabs)

9306

In [ ]:
from gensim import corpora, models

# Example corpus of documents (list of lists of tokens)
with open('./Data/congressional_bill_train_processed.pkl', 'rb') as inp:
    data = pickle.load(inp)
    corpus = data['datawords_nonstop']

# Create a dictionary from the corpus
dictionary = corpora.Dictionary(corpus)

# Create a bag-of-words representation of the corpus
bow_corpus = [dictionary.doc2bow(doc) for doc in corpus]

# Train an LDA model on the bag-of-words corpus
lda_model = models.LdaModel(bow_corpus, num_topics=13, id2word=dictionary, passes=10)



Word Prior Probability for 'college': 0.0000


In [ ]:
def get_prior(target_word):
    # The word for which you want to get the word prior probability
    # target_word = "veteran"

    # Get the word prior probability for the target word
    word_id = dictionary.token2id[target_word]
    word_prior_prob = sum(prob for _, prob in lda_model.get_term_topics(word_id))

    print(f"Word Prior Probability for '{target_word}': {word_prior_prob:.4f}")

In [ ]:
for word in corpus[0]:
    get_prior(word)

Word Prior Probability for 'authorize': 0.0886
Word Prior Probability for 'study': 0.0000
Word Prior Probability for 'feasibility': 0.0000
Word Prior Probability for 'desirability': 0.0000
Word Prior Probability for 'establish': 0.0602
Word Prior Probability for 'national': 0.0994
Word Prior Probability for 'recreation': 0.0000
Word Prior Probability for 'area': 0.0113
Word Prior Probability for 'know': 0.0000
Word Prior Probability for 'santa': 0.0000
Word Prior Probability for 'margarita': 0.0000
Word Prior Probability for 'national': 0.0994
Word Prior Probability for 'recreation': 0.0000
Word Prior Probability for 'area': 0.0113
Word Prior Probability for 'area': 0.0113
Word Prior Probability for 'san': 0.0000
Word Prior Probability for 'diego': 0.0000
Word Prior Probability for 'county': 0.0145
Word Prior Probability for 'calif': 0.0000
Word Prior Probability for 'presently': 0.0000
Word Prior Probability for 'constitute': 0.0000
Word Prior Probability for 'camp': 0.0000
Word Prior